In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression
from sklearn.model_selection import train_test_split

# Load Boston Housing Dataset
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['MEDV'] = boston.target

# Introduce missing values artificially
df.loc[::10, 'RM'] = np.nan
df.loc[::15, 'AGE'] = np.nan

# Task 1: Impute missing values using mean
imputer_mean = SimpleImputer(strategy='mean')
df[['RM', 'AGE']] = imputer_mean.fit_transform(df[['RM', 'AGE']])

# Task 2: Impute categorical using most frequent
# Creating a sample Titanic-like dataset
data_cat = pd.DataFrame({
    'Sex': ['male', 'female', np.nan, 'male', 'female', 'female', np.nan],
    'Embarked': ['S', 'C', np.nan, 'S', 'Q', 'S', np.nan],
    'Age': [22, 35, np.nan, 28, 30, np.nan, 40]
})

imputer_mode = SimpleImputer(strategy='most_frequent')
data_cat[['Sex', 'Embarked']] = imputer_mode.fit_transform(data_cat[['Sex', 'Embarked']])

# Task 3: KNN Imputation
knn_imputer = KNNImputer(n_neighbors=3)
df_knn = df.copy()
df_knn[['RM', 'AGE']] = knn_imputer.fit_transform(df_knn[['RM', 'AGE']])

# Task 4: Standardization
scaler_std = StandardScaler()
df_std = pd.DataFrame(scaler_std.fit_transform(df.drop(columns='MEDV')), columns=df.columns[:-1])

# Task 5: Min-Max Scaling
scaler_mm = MinMaxScaler()
df_mm = pd.DataFrame(scaler_mm.fit_transform(df.drop(columns='MEDV')), columns=df.columns[:-1])

# Task 6: Robust Scaling
scaler_robust = RobustScaler()
df_robust = pd.DataFrame(scaler_robust.fit_transform(df.drop(columns='MEDV')), columns=df.columns[:-1])

# Task 7: Correlation Matrix and drop highly correlated features
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
df_corr_filtered = df.drop(columns=to_drop)

# Task 8: Mutual Information
X = df.drop(columns='MEDV')
y = df['MEDV']
mi_scores = mutual_info_regression(X, y)
mi_results = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Task 9: Variance Threshold
selector = VarianceThreshold(threshold=0.1)
X_var = selector.fit_transform(X)
selected_columns = X.columns[selector.get_support()]
df_variance_filtered = pd.DataFrame(X_var, columns=selected_columns)

# Output check (Optional print examples)
print("Top MI Features:\n", mi_results.head())
print("Dropped due to correlation:", to_drop)
print("Remaining after variance threshold:", list(selected_columns))


ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: